In [2]:
import nltk

In [3]:
from xml.dom import minidom
import pandas as pd
import numpy as np
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn import svm
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

import pickle

In [6]:
from gensim.models import Word2Vec
import time

In [4]:
train = pd.read_csv("train_clean_stem.csv", sep=';',index_col=0)

In [32]:
# Skip-gram model (sg = 1)
size = 1000
window = 3
min_count = 1
workers = 3
sg = 1

word2vec_model_file = 'model/' + 'word2vec_default.model'
start_time = time.time()
stemmed_tokens = pd.Series(train['commentaire_stemmed']).values
# Train the Word2Vec Model
w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model.save(word2vec_model_file)

Time taken to train word2vec model: 1896.7988848686218


In [7]:
sg_w2v_model = Word2Vec.load('model/word2vec_1000.model')

In [9]:
# Store the vectors for train data in following file
train_word2vec_filename = 'word2vec/' + 'train_review_word2vec.csv'
with open(train_word2vec_filename, 'w+') as word2vec_file:
    for index, row in train.iterrows():
        model_vector = (np.mean([sg_w2v_model.wv[token] for token in row['commentaire_stemmed']], axis=0)).tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(1000))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(1000)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

In [19]:
dev = pd.read_csv("dev_clean_stem.csv", sep=';',index_col=0)

In [20]:
# Store the vectors for train data in following file
dev_word2vec_filename = 'word2vec/' + 'dev_review_word2vec.csv'
with open(dev_word2vec_filename, 'w+') as word2vec_file:
    for index, row in dev.iterrows():
        model_vector = np.mean([sg_w2v_model.wv[token] for token in row['commentaire_stemmed'] if token in sg_w2v_model.wv.vocab], axis=0).tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(1000))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(1000)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

In [29]:
test = pd.read_csv("test_clean_stem.csv", sep=';',index_col=0)
test_word2vec_filename = 'word2vec/' + 'test_review_word2vec.csv'

In [26]:
# Store the vectors for train data in following file
with open(test_word2vec_filename, 'w+') as word2vec_file:
    for index, row in test.iterrows():
        model_vector = np.mean([sg_w2v_model.wv[token] for token in row['commentaire_stemmed'] if token in sg_w2v_model.wv.vocab], axis=0).tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(1000))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(1000)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# Load from the filename
test_word2vec_df = pd.read_csv(test_word2vec_filename)
#Initialize the model
test_predictions_word2vec = svc_word2vec.predict(test_word2vec_df)

In [31]:
f = open("result/results_svc_word2vec.txt", "a")

for i in range(len(test_predictions_word2vec)):
    f.write(test.iloc[i]['review_id'] + " " + str(test_predictions_word2vec[i]) + "\n")
    #print(test.iloc[i]['review_id']," ",str(n),"\n")
f.close()